# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv('../output_data/cities.csv')
city_data_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ushuaia,40,AR,1596393517,75,-54.80,-68.30,41.00,24.16
1,severo-kurilsk,100,RU,1596393517,84,50.68,156.12,52.56,8.50
2,east london,0,ZA,1596393517,22,-33.02,27.91,72.57,8.46
3,hermanus,0,ZA,1596393518,72,-34.42,19.23,61.00,3.00
4,jamestown,90,US,1596393518,78,42.10,-79.24,73.99,18.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# gmaps configuration
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = city_data_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Narrow down the cities to fit weather conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
narrow_city_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 10) & (city_data_df["Cloudiness"] == 0) & \
                                   (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80)].dropna()

narrow_city_df



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,east london,0,ZA,1596393517,22,-33.02,27.91,72.57,8.46
169,port alfred,0,ZA,1596393557,24,-33.59,26.89,72.95,8.95
199,soyo,0,AO,1596393564,85,-6.13,12.37,72.66,7.45
208,lac du bonnet,0,CA,1596393565,51,50.25,-96.06,73.00,5.99
284,eldorado,0,BR,1596393585,39,-23.79,-54.28,78.98,8.95
296,sanandaj,0,IR,1596393587,21,35.31,46.99,72.55,4.50
408,komsomolskiy,0,UZ,1596393617,41,40.43,71.72,77.00,6.93
421,dzaoudzi,0,YT,1596393620,73,-12.79,45.27,75.20,8.05
436,axim,0,GH,1596393623,78,4.87,-2.24,74.14,8.21
468,san jose,0,US,1596393522,41,37.34,-121.89,79.00,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Create a hotel_df
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,east london,ZA,-33.02,27.91,
169,port alfred,ZA,-33.59,26.89,
199,soyo,AO,-6.13,12.37,
208,lac du bonnet,CA,50.25,-96.06,
284,eldorado,BR,-23.79,-54.28,
296,sanandaj,IR,35.31,46.99,
408,komsomolskiy,UZ,40.43,71.72,
421,dzaoudzi,YT,-12.79,45.27,
436,axim,GH,4.87,-2.24,
468,san jose,US,37.34,-121.89,


In [15]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [16]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
#     time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 2: east london.
Closest hotel in east london is Premier Hotels & Resorts.
------------
Retrieving Results for Index 169: port alfred.
Closest hotel in port alfred is The Halyards Hotel.
------------
Retrieving Results for Index 199: soyo.
Closest hotel in soyo is Kinwica.
------------
Retrieving Results for Index 208: lac du bonnet.
Closest hotel in lac du bonnet is Wildrose B & B.
------------
Retrieving Results for Index 284: eldorado.
Closest hotel in eldorado is Villa Verde Hotel.
------------
Retrieving Results for Index 296: sanandaj.
Closest hotel in sanandaj is hedayat hotel.
------------
Retrieving Results for Index 408: komsomolskiy.
Closest hotel in komsomolskiy is Diyor hotel.
------------
Retrieving Results for Index 421: dzaoudzi.
Closest hotel in dzaoudzi is Hotel Maharajah Mayotte.
------------
Retrieving Results for Index 436: axim.
Closest hotel in axim is Ankobra Beach.
------------
Retrieving Results for Index 468: san jose.
Closest hote

In [18]:
hotel_df = narrow_city_df
hotel_df['Hotel Name'] = np.nan


i = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": "hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[i, -1] = name
    i = i + 1

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,east london,0,ZA,1596393517,22,-33.02,27.91,72.57,8.46,Premier Hotels & Resorts
169,port alfred,0,ZA,1596393557,24,-33.59,26.89,72.95,8.95,The Halyards Hotel
199,soyo,0,AO,1596393564,85,-6.13,12.37,72.66,7.45,Kinwica
208,lac du bonnet,0,CA,1596393565,51,50.25,-96.06,73.00,5.99,Wildrose B & B
284,eldorado,0,BR,1596393585,39,-23.79,-54.28,78.98,8.95,Villa Verde Hotel
296,sanandaj,0,IR,1596393587,21,35.31,46.99,72.55,4.50,hedayat hotel
408,komsomolskiy,0,UZ,1596393617,41,40.43,71.72,77.00,6.93,Diyor hotel
421,dzaoudzi,0,YT,1596393620,73,-12.79,45.27,75.20,8.05,Hotel Maharajah Mayotte
436,axim,0,GH,1596393623,78,4.87,-2.24,74.14,8.21,Ankobra Beach
468,san jose,0,US,1596393522,41,37.34,-121.89,79.00,1.99,DoubleTree by Hilton Hotel San Jose


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))